# 자동주문프로그램 :-)  

### 조현정 개인프로젝트 2021. 07.

In [2]:
from openpyxl import Workbook # 엑셀
from openpyxl import load_workbook # 엑셀
import smtplib # 메일
from email.message import EmailMessage # 메일
from math import ceil # 올림처리
import datetime # 시간

###### 로그인

wb = load_workbook("관리자.xlsx")
ws = wb["관리자"]

로그인ID = []
로그인PW = []
    
for row in ws.iter_rows(min_row=2):
    로그인ID.append(row[0].value) 
for row in ws.iter_rows(min_row=2):
    로그인PW.append(row[1].value) 
    
loginbreak1 = 0    
loginbreak2 = 0
while loginbreak1 == 0:
    로그인_ID = input("아이디를 입력해주세요(구글메일) : ")
    if 로그인_ID in 로그인ID:        
        while loginbreak2 == 0:
            로그인_PW = input("비밀번호를 입력해주세요(앱비밀번호) : ")
            if 로그인_PW in 로그인PW:
                if 로그인PW.index("{}".format(로그인_PW)) == 로그인ID.index("{}".format(로그인_ID)):
                    print("안녕하세요.")
                    loginbreak2 = 1
                    loginbreak1 = 1
                else:
                    print("비밀번호를 잘못 입력하셨습니다.")
            else:
                print("비밀번호를 잘못 입력하셨습니다.")
    else:
        print("없는 아이디입니다.")

###### 담당자 정보 불러오기

관리자=[]

for column in tuple(ws.columns):  
    관리자.append(column[로그인ID.index(로그인_ID)+1].value)

ID = 관리자[0]
PW = 관리자[1]
상호명 = 관리자[2]
담당자 = 관리자[3]
직급 = 관리자[4]
직통번호 = 관리자[5]
팩스 = 관리자[6]

###### 설정값 불러오기

wb = load_workbook("관리자.xlsx")
ws = wb["설정값"]

항목=[]
품목=[]
재고_유지=[]
사용량_1회=[]
발주단위수량=[]
가격=[]
거래처=[]
이메일=[]
사용단위=[]
발주단위=[]

for row in ws.iter_rows(min_row=2):
    항목.append(row[0].value)
for row in ws.iter_rows(min_row=2):
    품목.append(row[1].value)
for row in ws.iter_rows(min_row=2):
    재고_유지.append(row[2].value)
for row in ws.iter_rows(min_row=2):
    사용량_1회.append(row[3].value)
for row in ws.iter_rows(min_row=2):
    발주단위수량.append(row[4].value)
for row in ws.iter_rows(min_row=2):
    가격.append(row[5].value)
for row in ws.iter_rows(min_row=2):
    거래처.append(row[6].value)
for row in ws.iter_rows(min_row=2):
    이메일.append(row[7].value)
for row in ws.iter_rows(min_row=2):
    사용단위.append(row[8].value)
for row in ws.iter_rows(min_row=2):
    발주단위.append(row[9].value)

wb.close()

###### 소모품 사용량, 주문수량 계산 및 발주예정내역 작성

now=datetime.datetime.now()
day="{}_{}_{}".format(now.year,now.month,now.day) # 파일명용!
day1="{}년 {}월 {}일".format(now.year,now.month,now.day) # 발주서용!

wb = load_workbook("내역_{}.xlsx".format(day)) # 판매내역 엑셀파일 불러와서 데이터 정리
ws = wb.active
매출내역 = []
for row in ws.iter_rows(min_row=2):
    매출내역.append(row[1].value)

total = 0
발주리스트 = []

print("\n<{}> 발주예정 내역".format(day)) # 소모량 및 주문량 계산 및 발주예정내역 출력
for i in range(len(품목)):
    소모량 = 사용량_1회[i] * 매출내역[i]    
    재고 = 재고_유지[i] - 소모량
    주문량 = ceil((소모량) / 발주단위수량[i])
    주문예정가격 = 주문량 * 가격[i]
    if 주문량 >= 1:
        total+=주문예정가격
        개별발주=[품목[i],주문량,주문예정가격,거래처[i],이메일[i],가격[i],소모량,재고,발주단위수량[i],사용단위[i],발주단위[i]]
        발주리스트.append(개별발주)
        발주내역="\n- 품목 : {}, 소모량 : {}{}, 남은재고 : {}{}\n  주문예정량 : {}{}({}{}/1{}), 주문예정가격 : {:,}원"\
        .format(품목[i],소모량,사용단위[i],재고,사용단위[i],주문량,발주단위[i],발주단위수량[i],사용단위[i],발주단위[i],주문예정가격)
        print(발주내역)
print("\n총 주문 예정 가격 : {:,}원".format(total))

###### 발주여부 묻기

while 1:
    발주진행 = input("\n\n발주 진행하시겠습니까?\n1. 발주 / 2. 발주내역 수정 / 3. 취소 : ")

###### 취소선택

    if 발주진행 == '3':
        print("\n프로그램을 종료합니다.")
        break

###### 발주내역 수정 선택

    elif 발주진행 == '2':
        발주예정품목 = []
        for i in range(len(발주리스트)):
            발주예정품목.append(발주리스트[i][0])  
            
        발주수정1 = 0
        발주수정2 = 0
        while 발주수정1 == 0:
            수정_품목 = input("\n수정할 품목을 입력해주세요 : ") # 수정할 품목 입력
            if 수정_품목 in 발주예정품목:
                while 발주수정2 == 0:
                    수정_수량 = input("\n수량을 입력해주세요 : ") # 수정할 수량 입력
                    if 수정_수량.isdigit():
                        발주수정2=1
                        발주수정1=1
                    else:
                        print("\n숫자로 입력해주세요 : ")
            else:
                print("발주예정목록에 없는 품목입니다.")

        for i in range(len(발주리스트)):
            if 수정_품목 == 발주리스트[i][0]:
                발주리스트[i][1] = 수정_수량 # 입력받은 수량을 발주리스트에 덮어써서 수정
                발주리스트[i][2] = int(int(수정_수량) * 발주리스트[i][5]) # 주문예정가격 수정
        
        삭제_발주리스트=[] 
        for i in range(len(발주리스트)):
            if 발주리스트[i][1] == '0': 
                삭제_발주리스트.append(i)
        for i in 삭제_발주리스트:
            del 발주리스트[i] # 수정할 수량을 0으로 입력한 품목은 발주 목록에서 삭제        
        
        print("\n<{}> 발주예정 내역".format(day)) # 수정된 발주예정내역 출력
        for i in range(len(발주리스트)):
            수정발주내역="\n- 품목 : {}, 소모량 : {}{}, 남은재고 : {}{}\n  주문예정량 : {}{}({}{}/1{}), 주문예정가격 : {:,}원"\
            .format(발주리스트[i][0],발주리스트[i][6],발주리스트[i][9],발주리스트[i][7],발주리스트[i][9],발주리스트[i][1],\
                    발주리스트[i][10],발주리스트[i][8],발주리스트[i][9],발주리스트[i][10],발주리스트[i][2])
            print(수정발주내역)
        total=0
        for i in range(len(발주리스트)):
            total+=발주리스트[i][2]
        print("\n총 주문 예정 가격 : {:,}원".format(total))

###### 발주진행 선택

    elif 발주진행 == '1':

###### 발주서 작성

        for i in range(len(발주리스트)): 
            wb = load_workbook("발주서.xlsx") # 발주서 양식 열기
            ws = wb.active # 활성화된 Sheet

            ws["F7"] = "{}".format(상호명) # 회사명
            ws["F8"] = "{}".format(담당자) # 담당자명
            ws["F9"] = "{}".format(직급) # 직급
            ws["F10"] = "{}".format(day1) # 주문일자
            ws["R7"] = "{}".format(직통번호)
            ws["R8"] = "{}".format(팩스)
            ws["R9"] = "{}".format(로그인_ID)
            ws["D13"] = "{} ({}{}/1{})".format(발주리스트[i][0],발주리스트[i][8],발주리스트[i][9],발주리스트[i][10]) # 품명
            ws["M13"] = "{} {}".format(발주리스트[i][1],발주리스트[i][10]) # 수량
            ws["O13"] = "{}".format(발주리스트[i][5]) # 단가
            ws["S13"] = "{}".format(발주리스트[i][2]) # 금액
            ws["S17"] = "{}".format(발주리스트[i][2]) #합계
            ws["A25"] = "{}".format(day1) # 날짜
            ws["W30"] = "{}".format(담당자) # 발주자
            
            ws.protection.sheet = True # 시트보호(수정방지)
            ws.protection.enable()
            ws.protection.passwlrd = "{}".format(로그인_PW)
            wb.save("{}_{}발주서_{}.xlsx".format(상호명,발주리스트[i][0],day)) # 다른 이름으로 저장
        wb.close() # 파일 닫기     
        
###### 메일발송

        for i in range(len(발주리스트)):
        
            msg = EmailMessage()
            msg["Subject"] ="{} {} 발주요청서입니다.".format(상호명,발주리스트[i][0]) # 제목
            msg["From"] = '{}'.format(로그인_ID) # 보내는 사람
            msg["To"] = "{}".format(발주리스트[i][4]) # 받는 사람
            msg.set_content("""
                    안녕하세요.
                    {} {} {}입니다.
                    {} 발주서 보내드리오니, 검토 후 진행 부탁드립니다.
                    특이사항 있으시면 아래 번호로 연락 바랍니다.
                    감사합니다.\n\n\n\n\n
                    {} {}
                    Tel    : {}
                    fax    : {}
                    e-mail : {}""".format(상호명,담당자,직급,발주리스트[i][0],담당자,직급,직통번호,팩스,ID)) # 본문

            with open("{}_{}발주서_{}.xlsx".format(상호명,발주리스트[i][0],day), "rb") as f:
                msg.add_attachment(f.read(), maintype="application", subtype="octet-stream", filename=f.name) # 파일첨부(발주서)

            with smtplib.SMTP("smtp.gmail.com", 587) as smtp:
                smtp.ehlo() # 연결이 잘 수립되는지 확인
                smtp.starttls() # 모든 내용이 암호화되어 전송
                smtp.login('{}'.format(로그인_ID), '{}'.format(로그인_PW)) # 로그인 ID, PW(앱비밀번호)
                smtp.send_message(msg) # 메일발송
        
        print("\n요청하신 총 {}건의 발주요청을 완료하였습니다.\n프로그램을 종료합니다.".format(len(발주리스트)))
        break

    else:
        print("\n잘못 입력하셨습니다. 다시 선택해주세요.")
        continue    
        
        

아이디를 입력해주세요(구글메일) : johj921015@gmail.com
비밀번호를 입력해주세요(앱비밀번호) : oqfwgnpxxazexssq
안녕하세요.

<2021_8_14> 발주예정 내역

- 품목 : QMA, 소모량 : 4개, 남은재고 : 296개
  주문예정량 : 1박스(100개/1박스), 주문예정가격 : 200,000원

- 품목 : DNPH, 소모량 : 20개, 남은재고 : 230개
  주문예정량 : 1박스(50개/1박스), 주문예정가격 : 300,000원

- 품목 : TSA, 소모량 : 90개, 남은재고 : 110개
  주문예정량 : 2박스(50개/1박스), 주문예정가격 : 100,000원

- 품목 : PDA, 소모량 : 150개, 남은재고 : 50개
  주문예정량 : 3박스(50개/1박스), 주문예정가격 : 240,000원

총 주문 예정 가격 : 840,000원


발주 진행하시겠습니까?
1. 발주 / 2. 발주내역 수정 / 3. 취소 : 1


gaierror: [Errno 11001] getaddrinfo failed